# Validate BoM

This script reads in an excel file(s) from sharepoint that have previously been extracted from 3dx, processed to add the additional functions/metrics and passed to smartsheets.

It can process as many files as you want for as many product structures as you want and build a history of data quality.  Currently writing out to a file named the same as the input file + '_validated' suffixed to the end


In [52]:
import pandas as pd
import numpy as np
import os
import re
import io
import xlwings as xw
import openpyxl
from pathlib import Path
import argparse
import platform
import sys


function to determine whether we're running in Juypter notebook or as a command line script

In [53]:
def type_of_script():
    '''
        determine where this script is running
        return either jupyter, ipython, terminal
    '''
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'

determine the folder structure based on whether we're running on a test windows pc, in azure server, a mac, or in the real world against sharepoint - helps Mark test on different devices! 

In [54]:
def set_folder_defaults():
    if 'macOS' in platform.platform():
        # set some defaults for testing on mac
        download_dir = Path('/Users/mark/Downloads')
        user_dir = download_dir
        sharepoint_dir = download_dir

    elif 'Server' in platform.platform():
        # we're on the azure server (probably)
        user_dir = Path('Z:/python/FilesIn')

        download_dir = Path(user_dir)
        user_dir = download_dir
        sharepoint_dir = Path('Z:/python/FilesOut')

    elif os.getlogin() == 'mark_':
        # my test windows machine
        download_dir = Path('C:/Users/mark_/Downloads')
        user_dir = download_dir
        sharepoint_dir = download_dir        

    else:
        # personal one drive
        user_dir = 'C:/Users/USERNAME'

        # replace USERNAME with current logged on user
        user_dir = user_dir.replace('USERNAME', os.getlogin())

        # read in config file
        config = configparser.ConfigParser()
        config.read('user_directory.ini')

        # read in gm_dir and gm_docs from config file
        gm_dir = Path(config[os.getlogin().lower()]['gm_dir'])
        gm_docs = Path(config[os.getlogin().lower()]['gmt'])
        # this may find more than one sharepoint directory
        # sharepoint_dir = user_dir + "/" + gm_dir + "/" + gm_docs
        sharepoint_dir = Path(user_dir / gm_dir / gm_docs)

        # download_dir = os.path.join(sharepoint_dir, 'Data Shuttle', 'downloads')
        download_dir = Path(sharepoint_dir / 'Data Shuttle' / 'downloads')

    return sharepoint_dir, download_dir, user_dir

based on the folder defaults look for the files we're interested in

In [55]:
def find_files(download_dir):
    # find any changed files changed in past 2hrs in the downloads directory
    dirpath = download_dir
    files = []
    for p, ds, fs in os.walk(dirpath):
        for fn in fs:
            if 'Updated_' in fn:
                # was using this to filter what filenames to find
                filepath = os.path.join(p, fn)
                files.append(filepath)

    return files

# Makes without Buys
For each Make part there should be an assembly that needs at least one Buy part below it to make sense - if you're going to bake a cake, you need at least 1 ingredient!  If you're buying a cake, then you don't need anything else!

If a MAKE is not followed by a child BUY this is a problem

In [57]:
def check_make_no_buy(df):
    # if MAKE and there is no BUY below next row's part level less than or equal to current part level we have a MAKE without a BUY
    # df['PROVIDE'] = np.where(df['Source Code'].isin(['AIH','MIH','MOB']),'Make','Buy')
    make_no_buy = list(df[(df['Source Code'].isin(['AIH','MIH','MOB'])) & (df['Level'].shift(-1) <= df['Level'])].index)
    make_no_buy = sorted(make_no_buy)
    df['make_no_buy'] = np.where((df['Source Code'].isin(['AIH','MIH','MOB'])) & (df['Level'].shift(-1) <= df['Level']), True, False)

    return df, make_no_buy

# Parent Source Code

We will need to track what the source code of the parent part is and validate whether it is a correct scenario.

This interates through the dataframe and writes out to each row what the parent part (the level above this row's level) source code was

In [58]:
def parent_source_code(df):
    prev_level = 0

    level_source = {}

    for i, x in df.iterrows():
        # take the current level source and store it
        level_source[x['Level']] = x['Source Code']
        if ((x['Level'] >= 4)):
            df.loc[i, 'Parent Source Code'] = level_source[x['Level'] - 1]
    
    return df

# Source Code within parent checks

sc_check_list is a list of invalid scenarios we are checking for with syntax: SOURCE CODE_PARENT SOURCE CODE

A dataframe of invalid rows is written to dict_checks[sc_check] and a column is added to the end of the main dataframe with the sc_check as a column name holding true or false

In [59]:
def source_code_within_parent_checks(dict_checks, df):
    # check for combinations of source codes within a parent source that's not accepted
    sc_check_list = ['AIH_POA','BOP_FIP','FAS_FAS','FIP_FIP','FIP_FAS']
    
    for sc_check in sc_check_list:
        sc, parent_sc = sc_check.split('_')

        dict_checks[sc_check] = df[(df['Source Code'] == sc) & (df['Parent Source Code'] == parent_sc)]

        df[sc_check] = np.where((df['Source Code'] == sc) & (df['Parent Source Code'] == parent_sc), True, False)


    return dict_checks, df

# Level 4 Source Code Checks

level 4 (assembly level when first level = 0) should only have Source Code 'MIH' or 'AIH'

In [88]:
def check_level_4_source_code_checks(dict_checks, df):
    # level 4 can only be MIH or AIH
    dict_checks['Non_MIH_AIH_Level_4'] = df[(df['Level'] == 4) & (~df['Source Code'].isin(['MIH','AIH']))]
    df['Non_MIH_AIH_Level_4'] = np.where((df['Level'] == 4) & (df['Source Code'] == 'BOF'), True, False)

    return dict_checks, df

# Fasteners with wrong parent source code

Fasteners should only be within parents of 'FIP','AIH,'MIH'



In [89]:
def FAS_wrong_parent_source_code(dict_checks, df):
    # FAS can only be within a FIP, AIH or MIH parent
    dict_checks['FAS_Wrong_Parent_Source_code'] = df[(df['Source Code'] == 'FAS') & (~df['Parent Source Code'].isin(['FIP','AIH','MIH']))]
    df['FAS_Wrong_Parent_Source_code'] = np.where((df['Source Code'] == 'FAS') & (~df['Parent Source Code'].isin(['FIP','AIH','MIH'])), True, False)

    return dict_checks, df


# Fastener checks

Look for scenarios where a description says washer, bolt or grommet but the source code says 'BOF'.  

In [62]:
def fastener_checks(dict_checks, df):
    # All BOF records that are fasteners should be {FAS}teners in the BOMS
    # Part Description contains washer, bolt, grommet
    # Source code = "BOF"
    fastener_check_list = ['^washer|^bolt|^grommet']        

    dict_checks['FAS_as_BOF'] = df[(df['Description'].str.lower().str.contains('{}'.format(fastener_check_list))) & (df['Source Code'] == 'BOF')]
    df['FAS_as_BOF'] = np.where((df['Description'].str.lower().str.contains('{}'.format(fastener_check_list))) & (df['Source Code'] == 'BOF'), True, False)

    return dict_checks, df

# Filter check columns

For writing out to excel on separate sheets, only need to keep the pertinent columns

In [82]:
def filter_check_columns(dict_checks):
    # reduce the selection of columns used for writing out later
    check_columns = [
    'orig_sort',
    'Function Group',
    'System',
    'Sub System',
    'Level',
    'Title',
    'Revision',
    'Description',
    'Parent Part',
    'Source Code',
    'Quantity',
    'Parent Source Code'
    ]

    for key in dict_checks.keys():
        print (key)
        dict_checks[key] = dict_checks[key][check_columns]

    return dict_checks

# Write to excel

Call xlwings with your pre-prepared dictionary and write out many sheets to one excel file, naming the sheets whatever you called your dictionary keys

In [70]:
def write_to_xl(outfile, df_dict):
    import xlwings as xw
    with xw.App(visible=True) as app:
        try:
            wb = xw.Book(outfile)
            print ("writing to existing {}".format(outfile))
        except FileNotFoundError:
            # create a new book
            print ("creating new {}".format(outfile))
            wb = xw.Book()
            wb.save(outfile)

        for key in df_dict.keys():
            try:
                ws = wb.sheets.add(key)
            except Exception as e:
                print (e)
            
            ws = wb.sheets[key]

            table_name = key

            ws.clear()

            df = df_dict[key].set_index(list(df_dict[key])[0])
            if table_name in [table.df for table in ws.tables]:
                ws.tables[table_name].update(df)
            else:
                table_name = ws.tables.add(source=ws['A1'],
                                            name=table_name).update(df)
    wb.save(outfile)

# write out to excel using sub system

This was used previously (GMD) might be useful again so haven't removed, but not currently calling.

Writes out the checks to sheets filtered against the sub system - maybe useful if we wanted to give the problem rows to a team to manage

In [65]:
def write_to_xl_sub_system(dict_checks):
    sub_sys = dict_checks[check]['Sub System'].unique()
    sub_sys.sort()

    for s_sys in sub_sys:

        df_temp = dict_checks[check][dict_checks[check]['Sub System'] == s_sys]

        if df_temp.shape[0] > 0:
            df_temp.to_excel(writer, sheet_name=s_sys, index=False)

            ws = writer.sheets[s_sys]
            wb = writer.book

            excel_formatting.adjust_col_width_from_col(ws)

# Main Processing

This is where the processing begins, and where and in which order we call the functions defined above.  

In [90]:
if __name__ == '__main__':

    # for reading in multiple files

    # files = find_files()
    dict_df = {}

    filename = 'Updated_T48e-01-Z00005_2024-07-19.xlsx'
    sharepoint_dir, download_dir, user_dir = set_folder_defaults()

    file = Path(download_dir) / filename

    df = pd.DataFrame()

    with open(file, "rb") as f:
        # reading in the historic excel files
        df = pd.read_excel(f, parse_dates=True)
        f.close()

    df.reset_index(drop=False, inplace=True)
    df.rename(columns={'index':'bom_order'}, inplace=True)

    # add parent source code to each row for validation checks to come
    df = parent_source_code(df)

    # initialise a dictionary to store all the check output as dataframes
    dict_checks = {}

    # complete the source code with parent source code checks
    dict_checks, df = source_code_within_parent_checks(dict_checks, df)

    # check all level 4 have the correct source code
    dict_checks, df = check_level_4_source_code_checks(dict_checks, df)

    # check for FAS with the wrong source code
    dict_checks, df = FAS_wrong_parent_source_code(dict_checks, df)

    # complete the fasteners source code checks
    dict_checks, df = fastener_checks(dict_checks, df)

    # look for make assemblys with no parts to buy
    df, make_no_buy = check_make_no_buy(df)
    dict_checks['make_no_buy'] = df.loc[make_no_buy]

    # write out just the cols we need to report against
    dict_checks = filter_check_columns(dict_checks)

    # add the full df to the sheet
    dict_checks['BOM'] = df



AIH_POA
BOP_FIP
FAS_FAS
FIP_FIP
FIP_FAS
Non_MIH_AIH_Level_4
FAS_Wrong_Parent_Source_code
FAS_as_BOF
make_no_buy


### Release status checks
For part levels >=4 groupby part number and release status and report which parts have more than 1 release status - should be unique per part number

| 	|	|	|Function Group|	Part Level|	Issue Level|	row|
|---|---|---|--------------|--------------|------------|-------|
|Part Number|	Sub Group|	Release Status|||||				
|T33-A1117X	|A01-Structure Systems	|REL|	T33-BoM-XP	|7.0	|1.0	|617|
|||AWT	|T33-BoM-XP	|6.0	|1.0	|1284|
|T33-A1475	|A02-Panels & Closure Systems	|AWT	|T33-BoM-XP	|6.0	|1.0	|137|
|||REL	|T33-BoM-XP	|6.0	|1.0	|230|
|T33-A1476X	|A02-Panels & Closure Systems	|AWT	|T33-BoM-XP	|6.0	|1.0	|143|


This is written out to excel


### Write out the validation checks to excel

- collated bom multi status   
- wrong parent parts   
- collated and cleaned bom written out with cleaned part numbers, parent parts and release status attached   

This is where we can add a list of source code combinations to check as we find out they are not valid (mainly causing a problem for IFS)

### Write out source code checks to excel

In [92]:
# Write out to excel
pathfile = Path(file.name).stem
output_file = Path(sys.path[0]) / Path(pathfile + '_validated').with_suffix('.xlsx')
write_to_xl(output_file, dict_checks)

writing to existing c:\Users\mark_\Documents\GitHub\BOM_from_3DX\Updated_T48e-01-Z00005_2024-07-19_validated.xlsx
Sheet named 'AIH_POA' already present in workbook
Sheet named 'BOP_FIP' already present in workbook
Sheet named 'FAS_FAS' already present in workbook
Sheet named 'FIP_FIP' already present in workbook
Sheet named 'FIP_FAS' already present in workbook
Sheet named 'Non_MIH_AIH_Level_4' already present in workbook
Sheet named 'FAS_Wrong_Parent_Source_code' already present in workbook
Sheet named 'FAS_as_BOF' already present in workbook
Sheet named 'make_no_buy' already present in workbook
Sheet named 'BOM' already present in workbook


In [86]:
for key in dict_checks.keys():
    print (key)
    print (dict_checks[key].orig_sort.head())

AIH_POA
363      933
388      965
647     1311
2747    4957
2749    4959
Name: orig_sort, dtype: object
BOP_FIP
Series([], Name: orig_sort, dtype: object)
FAS_FAS
297     496
316     541
405     995
427    1023
428    1029
Name: orig_sort, dtype: object
FIP_FIP
Series([], Name: orig_sort, dtype: object)
FIP_FAS
Series([], Name: orig_sort, dtype: object)
FAS_as_BOF
11    10
27    39
31    52
37    66
38    67
Name: orig_sort, dtype: object
make_no_buy
23      35
78     159
95     179
100    184
102    186
Name: orig_sort, dtype: object
BOM
0    percent_missing
1                  0
2                  1
3                  2
4                  3
Name: orig_sort, dtype: object


In [ ]:
import xlwings as xw
from openpyxl.utils.cell import get_column_letter

make_filename = os.path.join(sharepoint_dir, project, project + '_MakeBuyQuery.xlsm')

with xw.App():
    try:
        wb = xw.Book(make_filename)
    except:
        wb = xw.Book()
    
    ws = wb.sheets[0]

    startrow=0

    ws.range("A:XX").clear()

    ws['A1'].options(pd.DataFrame, header=1, index=False).value=existing_bom

    last_col_letter = get_column_letter(existing_bom.shape[1])


    for row in make_no_buy:
        xw.Range('A{}:{}{}'.format(row + 2, last_col_letter, row + 2)).color = (69, 165, 237)

    ws.tables.add(ws.used_range, name="a_table")

    # autofit the columns
    ws.autofit('c')
    wb.save(make_filename)


NameError: name 'project' is not defined

# Multiple Source Codes

In [ ]:
def multi_source_code(df):

    unstacked = df.groupby(['Part Number','Issue Level','Function Group','Sub Group','Source Code']).size().unstack()

    # find number of columns dynamically, as number of unique status controls the number of columns
    expected_status_count = len(unstacked.columns) - 1
    unstacked2 = unstacked[unstacked.isna().sum(axis=1)!=expected_status_count]
    unstacked2


    multi_sc = unstacked2.reset_index().fillna('')

    make_sc_cols = ['AIH','MIH','MOB']

    first_cols = ['Part Number', 'Issue Level', 'Function Group', 'Sub Group']

    cols_to_order = first_cols + make_sc_cols
    sc_ordered_cols = cols_to_order + (multi_sc.columns.drop(cols_to_order).tolist())

    multi_sc = multi_sc[sc_ordered_cols]

    return multi_sc


In [ ]:
multi_sc = multi_source_code(existing_bom)

In [ ]:
import xlwings as xw

multi_sc_filename = os.path.join(base, '{}-Multi-Source-Codes.xlsm'.format(project))

# wb = xw.Book(path)

# open file if it exists
try:
    wb = xw.Book(multi_sc_filename)
# otherwise create a new file    
except:
    wb = xw.Book()

ws = wb.sheets[0]

ws.clear_contents()

# autofit the columns
wb.sheets[ws].autofit('c')

startrow=0

ws['A1'].options(pd.DataFrame, header=1, index=False).value=multi_sc

# wb.save(multi_sc_filename) - this errors - not sure it's needed with autosave
